In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.6MB/s 
     |████████████████████████████████| 890kB 18.8MB/s 
     |████████████████████████████████| 3.0MB 29.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fd36ad74ba71a7ee62b82df433f1cf1dae1ff7d0eac4c8b4c3c749dddd46f790
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

questions = [
    ' What is the advantage of including onions and garlic in our diet ?']
paragraph = ''' Power foods are foods that provide rich levels of nutrients like fibre, potassium and minerals. With people becoming increasingly health conscious today, a lot of finess trainers encourage their clients to include these foods in their daily diet to increase muscle development. There are various ways of incorporating power foods in your daily diet. Of course, the key to enjoying power foods is proper preparation of these foods, the use of season-fresh foods, and indentifying your choice of flavour among power foods. '''

for question in questions:
  question_tokens = tokenizer.tokenize(question)
  paragraph_tokens = tokenizer.tokenize(paragraph)
  tokens = ['[CLS]'] + question_tokens + ['[SEP]'] + paragraph_tokens + ['[SEP]']
  input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_mask = [1] * len(input_word_ids)
  input_type_ids = [0] *(1 + len(question_tokens) + 1) + [1] * (len(paragraph_tokens) + 1)

  input_word_ids, input_mask, input_type_ids = map(lambda t: tf.expand_dims(
      tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_word_ids, input_mask, input_type_ids))
  outputs = model([input_word_ids, input_mask, input_type_ids])
  # using `[1:]` will enforce an answer. `outputs[0][0][0]` is the ignored '[CLS]' token logit
  short_start = tf.argmax(outputs[0][0][1:]) + 1
  short_end = tf.argmax(outputs[1][0][1:]) + 1
  answer_tokens = tokens[short_start: short_end + 1]
  answer = tokenizer.convert_tokens_to_string(answer_tokens)
  print(f'Question: {question}')
  print(f'Answer: {answer}')

Question:  What is the advantage of including onions and garlic in our diet ?
Answer: provide rich levels of nutrients like fibre , potassium and minerals
